In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt

# For financial data
import yahoo_fin
import yahoo_fin.stock_info as si
#import yahoo_fin.options as ops

# For NER/NLP
import spacy
nlp = spacy.load('en_core_web_sm')
import re
import codecs
import unidecode
import contractions

In [2]:
# Clean up text
# Source:  https://medium.com/the-innovation/can-we-actually-predict-market-change-by-analyzing-reddits-r-wallstreetbets-9d7716516c8e

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    
    # Expand all contractions in text
    expanded = contractions.fix(decoded)
    
    parsed = nlp(expanded)
    
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [3]:
""""
Desc: Extract the stock mentioned from the Reddit submission or comment.
Parameter: Submission or comment's text
output: list of entities labeled as ORG
Source: https://towardsdatascience.com/ner-for-extracting-stock-mentions-on-reddit-aa604e577be
"""
def get_company(text):
    
    BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda', 'treasury']
    
    # Read the text
    doc = nlp(text)
    
    # Empty list to store organization names
    orgs = []
    
    # For each entity in the doc.ents
    for entity in doc.ents:
        
        # If label_ is ORG and not in the BLACKLIST
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            
            # Add it to the list of organizations
            orgs.append(entity.text)
    
    # Convert to set() to remove duplicates, and then convert back to list
    orgs = list(set(orgs))
    
    # Each organization in orgs
    for i in range(len(orgs)):
        
        # If any of the companies in all_companies contain an organization mentioned, get the ticker
        if (all_companies["Name"].str.contains(orgs[i].lower(), regex = True).any()):
            orgs[i] = all_companies[all_companies["Name"].str.contains(orgs[i].lower())]['Symbol'].values[0]

        # Else if the ticker is mentioned
        elif (all_companies["Symbol"].str.match(orgs[i], case = False).any()):
            orgs[i] = all_companies[all_companies["Symbol"].str.match(orgs[i], case = False)]['Symbol'].values[0]

        # Else the orgs was not found
        else:
            orgs[i] = ""
    
    # Remove blank spaces in the list
    orgs[:] = [x for x in orgs if x.strip()]
    
    # If the list is empty
    if (len(orgs) == 0):
        orgs = ""
    
    return orgs

In [4]:
# Read all companies
all_companies = pd.read_csv("./Data/all_companies.csv")

# Function to do exact match for tickers
def get_ticker(text):
    
    # List to store the ticker
    ticker = []
    
    # For each ticker
    for i in range(len(all_companies["Symbol"])):
        
        # Check if each ticker is in the text (added a space on each side to make each ticker unique)
        if (all_companies["Symbol"][i].center(len(all_companies["Symbol"][i])+2) in text):
            
            # Add it to the list of tickers if it is found
            ticker.append(all_companies["Symbol"][i])
    
    return ticker

In [5]:
# Read the comments data
ci = pd.read_csv("./Data/comment_aa_info.csv", 
                 usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                 low_memory = False)

ci2 = pd.read_csv("./Data/comment_ab_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci3 = pd.read_csv("./Data/comment_ac_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci4 = pd.read_csv("./Data/comment_ad_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

ci5 = pd.read_csv("./Data/comment_ae_info.csv",
                  usecols = ["body", "date", "comment_id", "score", "submission_id", "number_of_replies", "total_awards"],
                  low_memory = False)

display(ci)

,body,date,comment_id,score,submission_id,number_of_replies,total_awards
0,I gotta say I'm impressed ot stayed above 15 a...,1.546349e+09,ed0exiy,16,abfam2,1,0
1,And it will do nothing or drop the entire year...,1.546430e+09,ed2nm06,2,abfam2,0,0
2,I was all in on amd from Nov 2017 to like Feb ...,1.546368e+09,ed0y55b,1,abfam2,0,0
3,Anyone selling puts on AMD here? I was consid...,1.546381e+09,ed1fwek,1,abfam2,1,0
4,Most people here aren't allowed by law to talk...,1.546320e+09,eczukex,8,abfckx,1,0
...,...,...,...,...,...,...,...
1690205,Ban,1.594350e+09,fxhs5or,1,hohaoi,0,0
1690206,bull market makes you money.\n\nbear market ma...,1.594351e+09,fxhsvtu,8,hohcin,0,0
1690207,"Idk why but you make me wanna do the same put,...",1.594351e+09,fxht27h,5,hohcin,0,0
1690208,Flare checks doubt,1.594351e+09,fxhta8e,2,hohcin,0,0


In [6]:
# Read the comment author info data
cai = pd.read_csv("./Data/comment_author_aa_info.csv", 
                  usecols = ["author_commentkarma", "author_id", "author_name", "comment_id", "submission_id"],
                  low_memory = False)

cai2 = pd.read_csv("./Data/comment_author_ab_info.csv", 
                   usecols = ["author_commentkarma", "author_id", "author_name", "comment_id", "submission_id"],
                   low_memory = False)

cai3 = pd.read_csv("./Data/comment_author_ac_info.csv", 
                   usecols = ["author_commentkarma", "author_id", "author_name", "comment_id", "submission_id"],
                   low_memory = False)

cai4 = pd.read_csv("./Data/comment_author_ad_info.csv", 
                   usecols = ["author_commentkarma", "author_id", "author_name", "comment_id", "submission_id"],
                   low_memory = False)

cai5 = pd.read_csv("./Data/comment_author_ae_info.csv", 
                   usecols = ["author_commentkarma", "author_id", "author_name", "comment_id", "submission_id"],
                   low_memory = False)

display(cai)

,author_commentkarma,author_id,author_name,comment_id,submission_id
0,19144,w0rum,Bryant570,ed0exiy,abfam2
1,1450,2dgnq1pl,Rhsucks,ed2nm06,abfam2
2,5365,p9nan,NotThe0nion,ed0y55b,abfam2
3,14570,3j2cd,surfinbird,ed1fwek,abfam2
4,43980,2lf6bkvi,HatRemov3r,eczukex,abfckx
...,...,...,...,...,...
1454420,50219,6ih3uea3,4-eva-dickard,fxhs25s,hohaoi
1454421,12,ezsz59c,Falvear,fxhs5or,hohaoi
1454422,15945,1wm23ocg,Not_name_u_lookin_4,fxhsvtu,hohcin
1454423,4615,1alzw6hv,DungeonVig,fxht27h,hohcin


In [7]:
# Merge ci and cai
coms = ci.merge(cai, how="outer", on="comment_id")
coms2 = ci2.merge(cai2, how="outer", on="comment_id")
coms3 = ci3.merge(cai3, how="outer", on="comment_id")
coms4 = ci4.merge(cai4, how="outer", on="comment_id")
coms5 = ci5.merge(cai5, how="outer", on="comment_id")

In [8]:
# Filter out removed and deleted comments comments
coms = coms[coms['body'] != "[removed]"]
coms = coms[coms['body'] != "[deleted]"]

coms2 = coms2[coms2['body'] != "[removed]"]
coms2 = coms2[coms2['body'] != "[deleted]"]

coms3 = coms3[coms3['body'] != "[removed]"]
coms3 = coms3[coms3['body'] != "[deleted]"]

coms4 = coms4[coms4['body'] != "[removed]"]
coms4 = coms4[coms4['body'] != "[deleted]"]

coms5 = coms5[coms5['body'] != "[removed]"]
coms5 = coms5[coms5['body'] != "[deleted]"]

In [9]:
# check number of nans in each column for each data frame
print(coms.isna().sum())
print(coms2.isna().sum())
print(coms3.isna().sum())
print(coms4.isna().sum())
print(coms5.isna().sum())

body                       18
date                        0
comment_id                  0
score                       0
submission_id_x             0
number_of_replies           0
total_awards                0
author_commentkarma    175734
author_id              175734
author_name            175734
submission_id_y        175734
dtype: int64
body                       7
date                       1
comment_id                 0
score                      0
submission_id_x            0
number_of_replies          0
total_awards               1
author_commentkarma    66360
author_id              66360
author_name            66360
submission_id_y        66360
dtype: int64
body                      0
date                      0
comment_id                0
score                     0
submission_id_x           0
number_of_replies         0
total_awards              0
author_commentkarma    2492
author_id              2492
author_name            2492
submission_id_y        2492
dtype: int64
body

In [10]:
# Filter out comments that have NaN
coms = coms[~coms['body'].isna()]
coms2 = coms2[~coms2['body'].isna()]
coms3 = coms3[~coms3['body'].isna()]
coms4 = coms4[~coms4['body'].isna()]
coms5 = coms5[~coms5['body'].isna()]

coms2 = coms2[~coms2['date'].isna()]
coms2 = coms2[~coms2['total_awards'].isna()]

In [11]:
# Convert date to month-day-Year format as string
coms["date"] = coms["date"].map(dt.datetime.utcfromtimestamp)
coms2["date"] = coms2["date"].map(dt.datetime.utcfromtimestamp)
coms3["date"] = coms3["date"].map(dt.datetime.utcfromtimestamp)
coms4["date"] = coms4["date"].map(dt.datetime.utcfromtimestamp)
coms5["date"] = coms5["date"].map(dt.datetime.utcfromtimestamp)

In [12]:
# Reset all the indices
coms.reset_index(inplace = True)
coms2.reset_index(inplace = True)
coms3.reset_index(inplace = True)
coms4.reset_index(inplace = True)
coms5.reset_index(inplace = True)

In [19]:
# Convert the date column into string and reorder it as month, day, year
for i in range(len(coms["date"])):
    coms["date"][i] = coms["date"][i].strftime("%m/%d/%Y")

for j in range(len(com["date"])):
    coms["date"][i] = (coms["date"][i] + "/")[5:] + (coms["date"][i][:4])

C:\Users\denni\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [ ]:
for j in range(len(coms2["date"])):
    coms2["date"][j] = coms2["date"][j].strftime("%m/%d/%Y")
    coms2["date"][j] = (coms2["date"][j] + "/")[5:] + (coms2["date"][j][:4])

for k in range(len(coms3["date"])):
    coms3["date"][k] = coms3["date"][k].strftime("%m/%d/%Y")
    coms3["date"][k] = (coms3["date"][k] + "/")[5:] + (coms3["date"][k][:4])

for l in range(len(coms4["date"])):
    coms4["date"][l] = coms4["date"][l].strftime("%m/%d/%Y")
    coms4["date"][l] = (coms4["date"][l] + "/")[5:] + (coms4["date"][l][:4])

for m in range(len(coms5["date"])):
    coms5["date"][m] = coms5["date"][m].strftime("%m/%d/%Y")
    coms5["date"][m] = (coms5["date"][m] + "/")[5:] + (coms5["date"][m][:4])

In [ ]:
# Remove all rows with 2021
coms = coms[~(coms.date.str.contains("2021"))]
coms2 = coms2[~(coms2.date.str.contains("2021"))]
coms3 = coms3[~(coms3.date.str.contains("2021"))]
coms4 = coms4[~(coms4.date.str.contains("2021"))]
coms5 = coms5[~(coms5.date.str.contains("2021"))]

print(len(coms), len(coms2), len(coms3), len(coms4), len(coms5))